In [11]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs

In [12]:
deflector_source_pairs = [(21, 22), (21, 23), (21, 24), (23, 24), (23, 25), (23, 26), (24, 25), (24, 26), (24, 27), (25, 26), (25, 27), (25, 28), (26, 27), (26, 28), (26, 29), (27, 28), (27, 29), (27, 30)]

snr_array, det_array, num_det_array, num_tot_array = [], [], [], []

for deflector_cut, source_cut in tqdm(deflector_source_pairs):
    output_dir = os.path.join(config.machine.data_dir, 'survey_params', f'deflector_{deflector_cut}_source_{source_cut}')

    pickles_detectable_gglenses = glob(f'{output_dir}/detectable_gglenses_*.pkl')
    detectable_gglenses = []

    for p in pickles_detectable_gglenses:
        gglenses = util.unpickle(p)
        detectable_gglenses.extend(gglenses)

    # print(f'{len(detectable_gglenses)} detectable lens(es)')

    # det_array.append(detectable_gglenses)
    num_det_array.append(len(detectable_gglenses))

    pickles_complete_snr_lists = glob(f'{output_dir}/complete_snr_list_*.pkl')
    complete_snr_lists = []

    for p in pickles_complete_snr_lists:
        snr_list = util.unpickle(p)
        complete_snr_lists.extend(snr_list)

    # snr_array.append(complete_snr_lists)
    num_tot_array.append(len(complete_snr_lists))

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:05<00:00,  3.26it/s]


In [13]:
for pair, num_det in zip(deflector_source_pairs, num_det_array):
    print(f'{pair}: {num_det} detectable lenses')

(21, 22): 30 detectable lenses
(21, 23): 29 detectable lenses
(21, 24): 25 detectable lenses
(23, 24): 40 detectable lenses
(23, 25): 32 detectable lenses
(23, 26): 21 detectable lenses
(24, 25): 24 detectable lenses
(24, 26): 20 detectable lenses
(24, 27): 16 detectable lenses
(25, 26): 23 detectable lenses
(25, 27): 21 detectable lenses
(25, 28): 19 detectable lenses
(26, 27): 29 detectable lenses
(26, 28): 0 detectable lenses
(26, 29): 0 detectable lenses
(27, 28): 0 detectable lenses
(27, 29): 0 detectable lenses
(27, 30): 0 detectable lenses


In [14]:
for pair, num_det, num_tot in zip(deflector_source_pairs, num_det_array, num_tot_array):
    try:
        print(f'{pair}: {num_det}/{num_tot} = {100 * num_det/num_tot:.2f}%')
    except ZeroDivisionError:
        pass

(21, 22): 30/487 = 6.16%
(21, 23): 29/2435 = 1.19%
(21, 24): 25/6186 = 0.40%
(23, 24): 40/12165 = 0.33%
(23, 25): 32/24427 = 0.13%
(23, 26): 21/38315 = 0.05%
(24, 25): 24/26501 = 0.09%
(24, 26): 20/41584 = 0.05%
(24, 27): 16/57475 = 0.03%
(25, 26): 23/41882 = 0.05%
(25, 27): 21/58776 = 0.04%
(25, 28): 19/63850 = 0.03%
(26, 27): 29/59221 = 0.05%
